<a href="https://colab.research.google.com/github/SEKI-YUTA/ZARD_Information_Collect/blob/main/ZARD%E3%81%AE%E3%82%B0%E3%83%83%E3%82%BA%E3%83%87%E3%83%BC%E3%82%BF%E3%81%AE%E5%8F%8E%E9%9B%86.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# スクレイピングに必要なパッケージをインストール
!pip install requests beautifulsoup4

# Googleドライブにマウント
from google.colab import drive
drive.mount('/content/drive')

zardFolderPath = "drive/MyDrive/ZARD/"
goodsFolderName = "goodsData"
goodsImageFolderName = "image"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import datetime
import os
from os import name
from dataclasses import dataclass, field, asdict

@dataclass
class GoodsItem:
  id: int
  name: str
  imageLink: str
  purchaseLink: str
  price: int
  description: int

@dataclass
class Genre:
  id: int
  name: str
  items: [GoodsItem]

@dataclass
class GoodsData:
  genres: [Genre]



# 現在日時の文字列を返す
def getTimeStr():
  dt_now = datetime.datetime.now()
  # print(dt_now)
  time = dt_now.strftime('%Y年%m月%d日_%H:%M:%S')
  # print(time)
  return time

# 現在時刻でフォルダを作成する関数
def createFolder(path):
  try:
      # フォルダを作成する
      os.makedirs(path)
      print(f"Folder '{path}' created successfully.")
  except FileExistsError:
      # フォルダがすでに存在する場合
      print(f"Folder '{path}' already exists.")

def replaceCharForJson(str):
  return str.replace("\'", "\"").replace("True", "true").replace("False", "false")


In [ ]:
import time
import requests as re
from bs4 import BeautifulSoup


header = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3864.0 Safari/53"
}

# 各アイテムの詳細ページから価格を取得する
def getPriceFromDetailPage(detailURL: str):
  res = re.get(detailURL, headers = header)
  if res.history:
    print("redirect occured! \(detailURL)")
  soup = BeautifulSoup(res.text, "html.parser")
  price = soup.find


In [ ]:
import time
import requests as re
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import json


header = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3864.0 Safari/53"
}

musingHomeUrl = "https://musing.jp/"
zardShopBaseUrl = "https://musing.jp/shop/zard/"
genreFrameClass = "itemwaku_all"
genreNameClass = "zard_title"
goodsClass = "item_zard"
goodsNameClass = "item_zard_txt"
goodsImageLinkClass = "fancybox"
saveGoodsImage = True

res = re.get(zardShopBaseUrl, headers = header)
soup = BeautifulSoup(res.text, "html.parser")
allGenreFrame = soup.find_all(
    True,
    attrs={
        'class': genreFrameClass
    }
)

genreId = 0
itemId = 0

imageUrlDict = {}
genreList = []

print(len(allGenreFrame))

for genreFrameIndex in range(0, len(allGenreFrame), 1):
  genreId = genreId + 1
  print(genreFrameIndex)
  frame = allGenreFrame[genreFrameIndex]
  genreName = frame.find(
      "div",
      attrs={
          'class': genreNameClass
      }
  ).text
  goodsList = frame.find_all(
      True,
      attrs={
          'class': goodsClass
      }
  )
  # print(genreName)
  genreGoodsList = []
  # print(len(goodsList))
  for goodsIndex in range(0, len(goodsList), 1):
    itemId = itemId + 1
    goods = goodsList[goodsIndex]
    goodsName = goods.find(
        "div",
        attrs={
            'class': goodsNameClass
        }
    ).text
    # print(goodsName)
    imageLink = ""
    imageLinkObject = goods.find(
        "a",
        attrs={
            'class': goodsImageLinkClass
        }
    )
    if imageLinkObject is None:
      imageLink = urljoin(musingHomeUrl, goods.find(
          "img",
          attrs={}
      ).get("src"))
    else:
      imageLink = urljoin(zardShopBaseUrl, imageLinkObject.get("href"))
    purchaseLink = goods.find(
        lambda tag: tag.name == "a" and "fancybox" not in tag.get("class", []) and tag.has_attr("href")
    ).get("href")
    if not purchaseLink.startswith("https://www.bgv.jp"):
      purchaseLink = urljoin(musingHomeUrl, purchaseLink)
    imageUrlDict["imageId" + str(itemId)] = imageLink
    # print(imageLink)
    # print(purchaseLink)
    # 詳細ページの構造的に抜き出しめんどくさそうだから一旦やらない
    # getPriceFromDetailPage(purchaseLink)
    goodsObj = GoodsItem(
        itemId, goodsName, imageLink, purchaseLink, 0, ""
    )
    genreGoodsList.append(goodsObj)
  # print("=================")
  genreList.append(
      Genre(
          genreId, genreName, genreGoodsList,
      )
  )

goodsData = GoodsData(genreList)
goodsDataDict = asdict(goodsData)
goodsJson = json.dumps(goodsDataDict, ensure_ascii=False, indent=2)
# replaced = replaceCharForJson(str(asdict(goodsData)))
# print(goodsJson)

saveFolderRootPath = zardFolderPath + goodsFolderName + "/" + getTimeStr()
createFolder(saveFolderRootPath)
with open(saveFolderRootPath + "/goods.json", "w", encoding="UTF-8") as writer:
  writer.write(str(goodsJson))

imageSaveFolder = saveFolderRootPath + "/" + goodsImageFolderName
print(imageSaveFolder)
createFolder(imageSaveFolder)

for imageName, url in imageUrlDict.items():
  imageRes = re.get(url)
  with open(imageSaveFolder + "/" + imageName + ".png", 'wb') as image:
    image.write(imageRes.content)


12
0
1
2
3
4
5
6
7
8
9
10
11
Folder 'drive/MyDrive/ZARD/goodsData/2024年11月18日_15:21:49' created successfully.
drive/MyDrive/ZARD/goodsData/2024年11月18日_15:21:49/image
Folder 'drive/MyDrive/ZARD/goodsData/2024年11月18日_15:21:49/image' created successfully.
